Genetic Algorithms(John Holland) - based on the concepts of natural selection and genetics. GAs are a subset of a much larger branch of computation known as Evolutionary Computation.


We have a pool or a population of possible solutions to the given problem

They undergo recombination and mutation (like in natural genetics), producing new children, and the process is repeated over various generations

Each individual candidate solution  is assigned a fitness value and the fitter individuals are given a higher chance to mate and yield more “fitter” individuals (Darwinian Theory of “Survival of the Fittest”).


We keep “evolving” better individuals or solutions over generations, till we reach a stopping criterion





**Population** − It is a subset of all the possible (encoded) solutions to the given problem. The population for a GA is analogous to the population for human beings except that instead of human beings, we have Candidate Solutions representing human beings.


**Chromosomes** − A chromosome is one such solution to the given problem.

**Gene** − A gene is one element position of a chromosome.

**Allele** − It is the value a gene takes for a particular chromosome



![alt text](https://www.tutorialspoint.com/genetic_algorithms/images/basic_structure.jpg "Genetic Algorithm flow")


*   start with an initial population
*   select parents for mating
*   generate new off-springs (crossover and mutation)



```
GA()
   initialize population
   find fitness of population
   
   while (termination criteria is reached) do
      parent selection
      crossover with probability pc
      mutation with probability pm
      decode and fitness calculation
      survivor selection
      find best
   return best
```







In [ ]:
%%writefile genetic.py
#All Imports
import random
import statistics
import sys
import time

class Chromosome:
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness

def _mutate(parent, geneSet, get_fitness):
    childGenes = parent.Genes[:]
    index = random.randrange(0, len(parent.Genes))
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness)

def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)

def _get_improvement(new_child, generate_parent):
    bestParent = generate_parent()
    yield bestParent
    while True:
        child = new_child(bestParent)
        if bestParent.Fitness > child.Fitness:
            continue
        if not child.Fitness > bestParent.Fitness:
            bestParent = child
            continue
        yield child
        bestParent = child

def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    # get_fitness function
    # targetLen - 16 # size*2
    # optimalFitness - 0
    # geneSet - [0, 1, 2, 3, 4, 5, 6, 7]
    # display - function
    #random.seed()

    def fnMutate(parent):
        #print("parent",parent)
        return _mutate(parent, geneSet, get_fitness)

    def fnGenerateParent():
        return _generate_parent(targetLen, geneSet, get_fitness)

    for improvement in _get_improvement(fnMutate, fnGenerateParent):
        display(improvement)
        if not optimalFitness > improvement.Fitness:
            return improvement

Overwriting genetic.py


In [ ]:
%%writefile ga_nqueen.py

import datetime
import unittest

import genetic

class Fitness:
    def __init__(self, total):
        self.Total = total

    def __gt__(self, other):
        return self.Total < other.Total

    def __str__(self):
        return "{}".format(self.Total)

class Board:
    def __init__(self, genes, size):
        board = [['.'] * size for _ in range(size)]
        for index in range(0, len(genes), 2):
            row = genes[index]
            column = genes[index + 1]
            board[column][row] = 'Q'
        self._board = board

    def get(self, row, column):
        return self._board[column][row]

    def print(self):
        # 0,0 prints in bottom left corner
        for i in reversed(range(len(self._board))):
            print(' '.join(self._board[i]))


def display(candidate, startTime, size):
    timeDiff = datetime.datetime.now() - startTime
    board = Board(candidate.Genes, size)
    board.print()
    print("{}\t- {}\t{}".format(
        ' '.join(map(str, candidate.Genes)),
        candidate.Fitness,
        timeDiff))
    

def get_fitness(genes, size):
    board = Board(genes, size)
    rowsWithQueens = set()
    colsWithQueens = set()
    northEastDiagonalsWithQueens = set()
    southEastDiagonalsWithQueens = set()
    for row in range(size):
        for col in range(size):
            if board.get(row, col) == 'Q':
                rowsWithQueens.add(row)
                colsWithQueens.add(col)
                northEastDiagonalsWithQueens.add(row + col)
                southEastDiagonalsWithQueens.add(size - 1 - row + col)
    total = size - len(rowsWithQueens) \
            + size - len(colsWithQueens) \
            + size - len(northEastDiagonalsWithQueens) \
            + size - len(southEastDiagonalsWithQueens)
    #print(size,len(rowsWithQueens),len(colsWithQueens),len(northEastDiagonalsWithQueens),len(southEastDiagonalsWithQueens),total,sep=" : ")
    return Fitness(total)

#Debuging
db="debugging "

class EightQueensTests(unittest.TestCase):
    def test(self, size=8):
        def fnDisplay(candidate_chromosome):
            display(candidate_chromosome, startTime, size)
            #print(candidate_chromosome.Genes,candidate_chromosome.Fitness)

        def fnGetFitness(genes):
            #print(genes)
            return get_fitness(genes, size)


        geneset = [i for i in range(size)]
        # print(db,1,geneset)

        startTime = datetime.datetime.now()        

        optimalFitness = Fitness(0)
        # print(db,2,optimalFitness)

        best_chromosome= genetic.get_best(fnGetFitness, 2 * size, optimalFitness, geneset, fnDisplay)
        #best - Chromosome object
        # print(db,3,best_chromosome,best_chromosome.Genes,best_chromosome.Fitness)

        self.assertTrue(not optimalFitness > best_chromosome.Fitness)

    # def test_benchmark(self):
    #     genetic.Benchmark.run(lambda: self.test(20))



if __name__ == '__main__':
    unittest.main()

Overwriting ga_nqueen.py


In [ ]:
%%shell
python ga_nqueen.py

. Q . . . . . .
Q Q . . . . . .
. . . . . . . .
. . . . . Q . .
Q . Q . . . . .
. . . . . . . Q
. . . . . . . .
. . . . . . . .
2 3 1 7 5 4 0 6 1 6 5 4 0 3 7 2	- 10	0:00:00.000132
. . . . . . . Q
Q Q . . . . . .
. . . . . . . .
. . . . . Q . .
Q . Q . . . . .
. . . . . . . Q
. . . . . . . .
. . . . . . . .
2 3 7 7 5 4 0 6 1 6 5 4 0 3 7 2	- 9	0:00:00.000316
. . . . . . . Q
Q Q . . . . . .
. . . . . . . .
Q . . . . Q . .
Q . Q . . . . .
. . . . . . . Q
. . . . . . . .
. . . . . . . .
2 3 7 7 5 4 0 6 1 6 0 4 0 3 7 2	- 7	0:00:00.000500
. . . . . . . Q
Q . . . . . . .
. . . . . . . .
. . . . . Q . Q
Q . Q . . . . .
. . . . . . . Q
. . . . . . . .
. . . . Q . . .
2 3 7 7 5 4 0 6 4 0 7 4 0 3 7 2	- 6	0:00:00.000718
. . . . . . . Q
Q . . . . . . .
. . . . . . . .
. . . . . . Q Q
Q . Q . . . . .
. . . . . Q . .
. . . . . . . .
. . . . Q . . .
2 3 7 7 6 4 0 6 4 0 7 4 0 3 5 2	- 5	0:00:00.001002
. . . . . . . Q
. . Q . . . . .
. . . . . . . .
. . . Q . Q . .
Q . . . . . . .
. . . Q . . . .
. . . . 